In [1]:
import pandas as pd
import numpy as np

import pickle

# gensim
from gensim import corpora, models, similarities, matutils

# sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# logging for gensim (set to INFO)
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

import nltk
import spacy

from textblob import TextBlob, Word

import re

from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity

Edited version of AOC to make it more organized

In [3]:
pd.set_option('display.max_colwidth', 400)  # or 199

Now, preprocessing (using same code as for kickstarter as baseline, come back here to tweak later)

Look into https://pypi.org/project/tweet-preprocessor/ for tweet processing later!

In [4]:
nlp = spacy.load('en', disable=['parser', 'ner'])
# nlp = spacy.load('en')

In [5]:
def pre_process(text):
    
    # lowercase
    text=text.lower()
    
    #remove tags
    text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)

    # remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",text)
    
    #lemmatize with Spacy
    doc = nlp(text)
    text = " ".join([token.lemma_ for token in doc])
    
    return text

In [6]:
tweet_stopwords = stopwords.words('english') + \
    ['rt', 'https', 'http', 'amp', 'via', 'one', 'around', 'would', 'let', 'could', 'going', 'like', 
     'get', 'may', 'says', 'say', 'make', 'based', 'even', 'another', 'completely', 'thanks', 'way', 
     'find', 'used', 'thing', '2019', 'see', 'need', 'know', 'knows', 'think', 'thinks', 'take', 'new', 
     'day', 'days', 'tucker', 'carlson', 'tuckercarlson', 'pron']

In [7]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [9]:
with open('/Users/robertpagano/src/metis_project_kojak/network_files/tucker/community_pickles_pre_nlp/df_0.pickle', 'rb') as f:
    df_0 = pickle.load(f)

with open('/Users/robertpagano/src/metis_project_kojak/network_files/tucker/community_pickles_pre_nlp/df_1.pickle', 'rb') as f:
    df_1 = pickle.load(f)
    
with open('/Users/robertpagano/src/metis_project_kojak/network_files/tucker/community_pickles_pre_nlp/df_2.pickle', 'rb') as f:
    df_2 = pickle.load(f)
    
with open('/Users/robertpagano/src/metis_project_kojak/network_files/tucker/community_pickles_pre_nlp/df_3.pickle', 'rb') as f:
    df_3 = pickle.load(f)
    
with open('/Users/robertpagano/src/metis_project_kojak/network_files/tucker/community_pickles_pre_nlp/df_4.pickle', 'rb') as f:
    df_4 = pickle.load(f)
    
with open('/Users/robertpagano/src/metis_project_kojak/network_files/tucker/community_pickles_pre_nlp/df_5.pickle', 'rb') as f:
    df_5 = pickle.load(f)
    
with open('/Users/robertpagano/src/metis_project_kojak/network_files/tucker/community_pickles_pre_nlp/df_6.pickle', 'rb') as f:
    df_6 = pickle.load(f)
    
with open('/Users/robertpagano/src/metis_project_kojak/network_files/tucker/community_pickles_pre_nlp/df_7.pickle', 'rb') as f:
    df_7 = pickle.load(f)
    
with open('/Users/robertpagano/src/metis_project_kojak/network_files/tucker/community_pickles_pre_nlp/df_8.pickle', 'rb') as f:
    df_8 = pickle.load(f)
    
with open('/Users/robertpagano/src/metis_project_kojak/network_files/tucker/community_pickles_pre_nlp/df_9.pickle', 'rb') as f:
    df_9 = pickle.load(f)
    
with open('/Users/robertpagano/src/metis_project_kojak/network_files/tucker/community_pickles_pre_nlp/df_10.pickle', 'rb') as f:
    df_10 = pickle.load(f)
    
with open('/Users/robertpagano/src/metis_project_kojak/network_files/tucker/community_pickles_pre_nlp/df_11.pickle', 'rb') as f:
    df_11 = pickle.load(f)
    
with open('/Users/robertpagano/src/metis_project_kojak/network_files/tucker/community_pickles_pre_nlp/df_12.pickle', 'rb') as f:
    df_12 = pickle.load(f)

In [10]:
df_0['final_text'] = np.where((df_0['retweet_text'].isnull() == False), df_0['retweet_text'], df_0['main_text'])
df_1['final_text'] = np.where((df_1['retweet_text'].isnull() == False), df_1['retweet_text'], df_1['main_text'])
df_2['final_text'] = np.where((df_2['retweet_text'].isnull() == False), df_2['retweet_text'], df_2['main_text'])
df_3['final_text'] = np.where((df_3['retweet_text'].isnull() == False), df_3['retweet_text'], df_3['main_text'])
df_4['final_text'] = np.where((df_4['retweet_text'].isnull() == False), df_4['retweet_text'], df_4['main_text'])
df_5['final_text'] = np.where((df_5['retweet_text'].isnull() == False), df_5['retweet_text'], df_5['main_text'])
df_6['final_text'] = np.where((df_6['retweet_text'].isnull() == False), df_6['retweet_text'], df_6['main_text'])
df_7['final_text'] = np.where((df_7['retweet_text'].isnull() == False), df_7['retweet_text'], df_7['main_text'])
df_8['final_text'] = np.where((df_8['retweet_text'].isnull() == False), df_8['retweet_text'], df_8['main_text'])
df_9['final_text'] = np.where((df_9['retweet_text'].isnull() == False), df_9['retweet_text'], df_9['main_text'])
df_10['final_text'] = np.where((df_10['retweet_text'].isnull() == False), df_10['retweet_text'], df_10['main_text'])
df_11['final_text'] = np.where((df_11['retweet_text'].isnull() == False), df_11['retweet_text'], df_11['main_text'])
df_12['final_text'] = np.where((df_12['retweet_text'].isnull() == False), df_12['retweet_text'], df_12['main_text'])

In [11]:
df_0['tweet_processed'] = df_0['final_text'].apply(lambda x:pre_process(x))
df_1['tweet_processed'] = df_1['final_text'].apply(lambda x:pre_process(x))
df_2['tweet_processed'] = df_2['final_text'].apply(lambda x:pre_process(x))
df_3['tweet_processed'] = df_3['final_text'].apply(lambda x:pre_process(x))
df_4['tweet_processed'] = df_4['final_text'].apply(lambda x:pre_process(x))
df_5['tweet_processed'] = df_5['final_text'].apply(lambda x:pre_process(x))
df_6['tweet_processed'] = df_6['final_text'].apply(lambda x:pre_process(x))
df_7['tweet_processed'] = df_7['final_text'].apply(lambda x:pre_process(x))
df_8['tweet_processed'] = df_8['final_text'].apply(lambda x:pre_process(x))
df_9['tweet_processed'] = df_9['final_text'].apply(lambda x:pre_process(x))
df_10['tweet_processed'] = df_10['final_text'].apply(lambda x:pre_process(x))
df_11['tweet_processed'] = df_11['final_text'].apply(lambda x:pre_process(x))
df_12['tweet_processed'] = df_12['final_text'].apply(lambda x:pre_process(x))

In [12]:
with open('/Users/robertpagano/src/metis_project_kojak/network_files/tucker/community_pickles_pre_nlp/preproccessed_text/df_0_preprocess.pickle', 'wb') as to_write:
    pickle.dump(df_0, to_write)

with open('/Users/robertpagano/src/metis_project_kojak/network_files/tucker/community_pickles_pre_nlp/preproccessed_text/df_1_preprocess.pickle', 'wb') as to_write:
    pickle.dump(df_1, to_write)

with open('/Users/robertpagano/src/metis_project_kojak/network_files/tucker/community_pickles_pre_nlp/preproccessed_text/df_2_preprocess.pickle', 'wb') as to_write:
    pickle.dump(df_2, to_write)

with open('/Users/robertpagano/src/metis_project_kojak/network_files/tucker/community_pickles_pre_nlp/preproccessed_text/df_3_preprocess.pickle', 'wb') as to_write:
    pickle.dump(df_3, to_write)

with open('/Users/robertpagano/src/metis_project_kojak/network_files/tucker/community_pickles_pre_nlp/preproccessed_text/df_4_preprocess.pickle', 'wb') as to_write:
    pickle.dump(df_4, to_write)

with open('/Users/robertpagano/src/metis_project_kojak/network_files/tucker/community_pickles_pre_nlp/preproccessed_text/df_5_preprocess.pickle', 'wb') as to_write:
    pickle.dump(df_5, to_write)

with open('/Users/robertpagano/src/metis_project_kojak/network_files/tucker/community_pickles_pre_nlp/preproccessed_text/df_6_preprocess.pickle', 'wb') as to_write:
    pickle.dump(df_6, to_write)

with open('/Users/robertpagano/src/metis_project_kojak/network_files/tucker/community_pickles_pre_nlp/preproccessed_text/df_7_preprocess.pickle', 'wb') as to_write:
    pickle.dump(df_7, to_write)

with open('/Users/robertpagano/src/metis_project_kojak/network_files/tucker/community_pickles_pre_nlp/preproccessed_text/df_8_preprocess.pickle', 'wb') as to_write:
    pickle.dump(df_8, to_write)

with open('/Users/robertpagano/src/metis_project_kojak/network_files/tucker/community_pickles_pre_nlp/preproccessed_text/df_9_preprocess.pickle', 'wb') as to_write:
    pickle.dump(df_9, to_write)

with open('/Users/robertpagano/src/metis_project_kojak/network_files/tucker/community_pickles_pre_nlp/preproccessed_text/df_10_preprocess.pickle', 'wb') as to_write:
    pickle.dump(df_10, to_write)

with open('/Users/robertpagano/src/metis_project_kojak/network_files/tucker/community_pickles_pre_nlp/preproccessed_text/df_11_preprocess.pickle', 'wb') as to_write:
    pickle.dump(df_11, to_write)

with open('/Users/robertpagano/src/metis_project_kojak/network_files/tucker/community_pickles_pre_nlp/preproccessed_text/df_12_preprocess.pickle', 'wb') as to_write:
    pickle.dump(df_12, to_write)


In [13]:
X_0 = df_1['tweet_processed']
X_1 = df_1['tweet_processed']
X_2 = df_2['tweet_processed']
X_3 = df_3['tweet_processed']
X_4 = df_4['tweet_processed']
X_5 = df_5['tweet_processed']
X_6 = df_6['tweet_processed']
X_7 = df_7['tweet_processed']
X_8 = df_8['tweet_processed']
X_9 = df_9['tweet_processed']
X_10 = df_10['tweet_processed']
X_11 = df_11['tweet_processed']
X_12 = df_12['tweet_processed']


In [14]:
tfidf_0 = TfidfVectorizer(stop_words=tweet_stopwords, token_pattern="\\b[a-z][a-z]+\\b", ngram_range=(1,2))
tfidf_1 = TfidfVectorizer(stop_words=tweet_stopwords, token_pattern="\\b[a-z][a-z]+\\b", ngram_range=(1,2))
tfidf_2 = TfidfVectorizer(stop_words=tweet_stopwords, token_pattern="\\b[a-z][a-z]+\\b", ngram_range=(1,2))
tfidf_3 = TfidfVectorizer(stop_words=tweet_stopwords, token_pattern="\\b[a-z][a-z]+\\b", ngram_range=(1,2))
tfidf_4 = TfidfVectorizer(stop_words=tweet_stopwords, token_pattern="\\b[a-z][a-z]+\\b", ngram_range=(1,2))
tfidf_5 = TfidfVectorizer(stop_words=tweet_stopwords, token_pattern="\\b[a-z][a-z]+\\b", ngram_range=(1,2))
tfidf_6 = TfidfVectorizer(stop_words=tweet_stopwords, token_pattern="\\b[a-z][a-z]+\\b", ngram_range=(1,2))
tfidf_7 = TfidfVectorizer(stop_words=tweet_stopwords, token_pattern="\\b[a-z][a-z]+\\b", ngram_range=(1,2))
tfidf_8 = TfidfVectorizer(stop_words=tweet_stopwords, token_pattern="\\b[a-z][a-z]+\\b", ngram_range=(1,2))
tfidf_9 = TfidfVectorizer(stop_words=tweet_stopwords, token_pattern="\\b[a-z][a-z]+\\b", ngram_range=(1,2))
tfidf_10 = TfidfVectorizer(stop_words=tweet_stopwords, token_pattern="\\b[a-z][a-z]+\\b", ngram_range=(1,2))
tfidf_11 = TfidfVectorizer(stop_words=tweet_stopwords, token_pattern="\\b[a-z][a-z]+\\b", ngram_range=(1,2))
tfidf_12 = TfidfVectorizer(stop_words=tweet_stopwords, token_pattern="\\b[a-z][a-z]+\\b", ngram_range=(1,2))


Community_0

In [15]:
bag_of_words_0 = tfidf_0.fit_transform(X_0)
lsa_0 = TruncatedSVD(20)
doc_topic_0 = lsa_0.fit_transform(bag_of_words_0)
lsa_0.explained_variance_ratio_

array([0.20472547, 0.0362846 , 0.03682357, 0.03117483, 0.02392398,
       0.02124166, 0.02186123, 0.01470538, 0.01401623, 0.01259035,
       0.0125832 , 0.01043019, 0.00917623, 0.00899052, 0.0075041 ,
       0.00741644, 0.00645229, 0.00625008, 0.00589862, 0.00550977])

In [16]:
display_topics(lsa_0, tfidf_0.get_feature_names(), 10)


Topic  0
exclusive sexual, exclusive, yet unearthed, audio co, unearthed audio, shocking yet, pageant shocking, shocking, usa pageant, yet

Topic  1
harass teen, teen lecture, pageant dressing, room roy, brag go, go pageant, ban shopping, dressing, contestant donald, mall

Topic  2
wife, good, herd probably, usa vulnerable, wife mean, mean wife, old miss, pretty good, wife dumb, probably pretty

Topic  3
co qcozayvyic, qcozayvyic, usa pageant, sexual comment, pageant shocking, shocking yet, shocking, comment miss, yet, sexual

Topic  4
rjw, child beauty, ob rjw, repeatedly talk, co ob, talk sex, ob, pageant co, sex child, repeatedly

Topic  5
co wubucc, vile sort, dtj, sort locker, language predatory, defend stop, stop co, wubucc, wubucc dtj, predatory disgusting

Topic  6
haodcdldqg, co haodcdldqg, bayer still, still advertise, advertise show, show co, advertise, bayer, still, show

Topic  7
news, fox news, fox, protest, pirro, protest place, host, news headquarters, headquarters, ou

Community_1

In [17]:
bag_of_words_1 = tfidf_1.fit_transform(X_1)
lsa_1 = TruncatedSVD(20)
doc_topic_1 = lsa_1.fit_transform(bag_of_words_1)
lsa_1.explained_variance_ratio_

array([0.20472547, 0.0362846 , 0.03682357, 0.03117483, 0.02392398,
       0.02124165, 0.02186122, 0.01470533, 0.01401628, 0.01259031,
       0.01258315, 0.01043041, 0.00917577, 0.00899023, 0.00750309,
       0.00741596, 0.00645184, 0.00625259, 0.00589639, 0.00549781])

In [18]:
display_topics(lsa_1, tfidf_1.get_feature_names(), 10)


Topic  0
exclusive sexual, exclusive, yet unearthed, audio co, unearthed audio, shocking yet, pageant shocking, shocking, usa pageant, yet

Topic  1
pageant dressing, moore ban, shopping mall, republican defend, lecture family, dressing, harass teen, ban shopping, contestant donald, brag go

Topic  2
wife, good, herd probably, usa vulnerable, wife mean, mean wife, old miss, pretty good, wife dumb, probably pretty

Topic  3
co qcozayvyic, qcozayvyic, usa pageant, sexual comment, pageant shocking, shocking yet, shocking, comment miss, yet, sexual

Topic  4
rjw, repeatedly talk, child beauty, ob rjw, co ob, talk sex, ob, pageant co, sex child, repeatedly

Topic  5
co wubucc, wubucc, sort locker, wubucc dtj, room language, vile sort, dtj, stop co, predatory disgusting, language predatory

Topic  6
co haodcdldqg, haodcdldqg, bayer still, still advertise, advertise show, show co, advertise, bayer, still, show

Topic  7
news, fox news, fox, protest, pirro, protest place, host, headquarters, 

Community_2

In [19]:
bag_of_words_2 = tfidf_2.fit_transform(X_2)
lsa_2 = TruncatedSVD(20)
doc_topic_2 = lsa_2.fit_transform(bag_of_words_2)
lsa_2.explained_variance_ratio_

array([0.0608889 , 0.02440875, 0.01730847, 0.01769216, 0.01744139,
       0.01402515, 0.01330772, 0.01352222, 0.01141726, 0.01062276,
       0.00976345, 0.00957896, 0.00825222, 0.0072184 , 0.00670939,
       0.00637342, 0.00536751, 0.00533393, 0.00511885, 0.00506078])

In [20]:
display_topics(lsa_2, tfidf_2.get_feature_names(), 10)


Topic  0
deplatform every, every effective, hunt attempt, effective conservative, attempt deplatform, indication officially, throe election, week serve, matter continue, serve nothing

Topic  1
call brian, stelter eunuch, co wdyknkxroh, wdyknkxroh, eunuch co, brian stelter, eunuch, stelter, brian, call

Topic  2
destroy, co ngcc, ngcc nk, ngcc, nk vt, destroy judgejeanine, nk, vt, judgejeanine co, co

Topic  3
fox news, fox, news, judge jeanine, jeanine, judge, hannity retweet, seem goal, brazile cancellation, co bogguio

Topic  4
people, praise kkk, byrd denigrate, denigrate tran, leader robert, describe bag, kkk leader, robert byrd, jap praise, campaign write

Topic  5
co xqb, jeanine happen, destroy judge, happen co, xqb, happen, judge jeanine, judge, jeanine, destroy

Topic  6
xyisuty, co xyisuty, xyisuty judgejeanine, foxnew number, number co, judgejeanine salemmediagrp, go foxnew, salemmediagrp, number, go

Topic  7
boy joke, frat, old frat, frat boy, give less, sh year, less sh

Community_3

In [21]:
bag_of_words_3 = tfidf_3.fit_transform(X_3)
lsa_3 = TruncatedSVD(20)
doc_topic_3 = lsa_3.fit_transform(bag_of_words_3)
lsa_3.explained_variance_ratio_

array([0.06696098, 0.05836745, 0.02872296, 0.02911257, 0.02159089,
       0.01490198, 0.01404814, 0.01328185, 0.01333077, 0.01281172,
       0.01198265, 0.01075585, 0.01015773, 0.00930102, 0.00892984,
       0.00812248, 0.00827696, 0.00812979, 0.00703821, 0.00693857])

In [22]:
display_topics(lsa_3, tfidf_3.get_feature_names(), 10)


Topic  0
retweet give, less sh, sh year, give less, sh, old frat, boy joke, frat boy, frat, less

Topic  1
people, campaign write, lead medium, leader robert, byrd denigrate, guy lead, kkk leader, denigrate tran, denigrate, describe bag

Topic  2
carusone, writing, week, medium matter, matter, medium, bigoted writing, semitic bigoted, dkz, writing medium

Topic  3
nothing indication, continue hunt, attempt deplatform, deplatform every, election medium, serve nothing, hunt attempt, throe, throe election, every effective

Topic  4
co apjtgsa, savagery, minute pure, apjtgsa va, matter two, pure savagery, apjtgsa, move goangelo, savagery move, burie medium

Topic  5
destruction exactly, warn last, congrat mmfa, hbk, ycp, ycp hbk, night rule, president congrat, mutually assured, mutually

Topic  6
co wdyknkxroh, wdyknkxroh, stelter eunuch, call brian, eunuch co, brian stelter, brian, eunuch, stelter, call

Topic  7
plan harass, home facebook, harass terrorize, facebook allow, sxei, allow b

Community_4

In [23]:
bag_of_words_4 = tfidf_4.fit_transform(X_4)
lsa_4 = TruncatedSVD(20)
doc_topic_4 = lsa_4.fit_transform(bag_of_words_4)
lsa_4.explained_variance_ratio_

array([0.16347441, 0.09288822, 0.05322989, 0.02847312, 0.02518633,
       0.02367749, 0.01912305, 0.01936446, 0.01867064, 0.01546717,
       0.01510128, 0.01410622, 0.01224245, 0.01095318, 0.01071494,
       0.00866776, 0.00861659, 0.00798693, 0.00806394, 0.00776563])

In [24]:
display_topics(lsa_4, tfidf_4.get_feature_names(), 10)


Topic  0
skbkckxj, icymi white, icymi, co skbkckxj, supremacist co, supremacist, white supremacist, white, co, trump

Topic  1
trump, russia, dossier, start, policy adviser, foreign, uk, russia dirt, russia investigation, hhvpzo

Topic  2
comment recently, upsetting comment, unearth different, deeply, deeply upsetting, radio tv, vtwi, different radio, number deeply, eblgx vtwi

Topic  3
call, jai ex, iqn, co jai, jai, ex iqn, ex, dig hour, world pain, sponge co

Topic  4
teen usa, miss teen, usa, miss, teen, usa pageant, shocking, shocking yet, pageant shocking, comment miss

Topic  5
mollyjongfast, piece, piece glamorous, co ov, wdr, ov wdr, ov, glamorous mollyjongfast, glamorous, fabulous

Topic  6
koch course, googlealert, bankroll, vxfrb, course co, vxfrb ni, ni googlealert, ni, co vxfrb, bankroll hateful

Topic  7
want co, syotaodv, excellent people, mollyjongfast far, piece mollyjongfast, co syotaodv, show want, favorite piece, far excellent, people watch

Topic  8
night, lkxs x

Community_5

In [25]:
bag_of_words_5 = tfidf_5.fit_transform(X_5)
lsa_5 = TruncatedSVD(20)
doc_topic_5 = lsa_5.fit_transform(bag_of_words_5)
lsa_5.explained_variance_ratio_

array([0.23132361, 0.21119127, 0.07700845, 0.06478183, 0.02779131,
       0.02628309, 0.01498442, 0.01149473, 0.01121925, 0.00943686,
       0.00830325, 0.00643777, 0.00614467, 0.00596982, 0.00509523,
       0.00489001, 0.00447072, 0.00418132, 0.00410397, 0.00381725])

In [26]:
display_topics(lsa_5, tfidf_5.get_feature_names(), 10)


Topic  0
suck, precisely, laura loomer, owen laura, sharpio, sharpio jacob, stop share, else suck, share shitty, precisely suck

Topic  1
shout want, neutral public, fuck vulnerable, endanger girl, public bathroom, girl shout, vulnerable underage, bathroom endanger, endanger, want fuck

Topic  2
maher gofundme, gofundme, gofundme support, support effort, effort, maher, bill maher, bill, support, promise bill

Topic  3
complicit, course, complicit course, vacuum, course candace, happen vacuum, complicit none, course ben, course complicit, pewdiepie complicit

Topic  4
chan, anyone, trump, right, talk radio, chan chan, redpill kotaku, redpill, radio donald, anyone manosphere

Topic  5
demonetize mean, something right, bulrcfl, video instantly, co bulrcfl, must something, instantly demonetize, demonetize, mean must, instantly

Topic  6
yet, pageant shocking, shocking yet, shocking, sexual comment, usa pageant, comment miss, pageant, yet unearthed, exclusive sexual

Topic  7
co hta, ydzko

Community_6

In [27]:
bag_of_words_6 = tfidf_6.fit_transform(X_6)
lsa_6 = TruncatedSVD(20)
doc_topic_6 = lsa_6.fit_transform(bag_of_words_6)
lsa_6.explained_variance_ratio_

array([0.37314481, 0.04842933, 0.0404977 , 0.03833522, 0.03033319,
       0.02026201, 0.01782307, 0.01647664, 0.01432949, 0.01356378,
       0.01214984, 0.00782997, 0.00760106, 0.00658863, 0.00649975,
       0.00643117, 0.00604095, 0.00544597, 0.00484676, 0.00456493])

In [28]:
display_topics(lsa_6, tfidf_6.get_feature_names(), 10)


Topic  0
blame, blame mikestuchbery, nt gun, islam blame, briannawu candace, online right, online, blame williamlegate, blame online, blame nt

Topic  1
old, medium matter, matter, medium, dumb shit, statute limitation, air medium, vastly, limitation, different conservative

Topic  2
frat boy, boy joke, boy, frat, old frat, give less, retweet give, less sh, sh, sh year

Topic  3
dirt, dig old, old dirt, dig, medium matter, matter, medium, dirt dig, finally co, co tsduhcnzly

Topic  4
use, use dirty, thermal energy, co nxvdtpom, china india, use thermal, nxvdtpom, auspol energypoverty, auspol, nxvdtpom theboltreport

Topic  5
limb, abe ztb, shitty howard, good joke, co abe, brown, world limb, stern wannabe, skinned year, show bad

Topic  6
anncoulter machinegunkelly, mrrogerstone, skynew piersmorgan, lisamurkowski waynedupreeshow, lisamurkowski, tgowdysc, tgowdysc genflynn, lindseygrahamsc politicalshort, anncoulter, ingrahamangle realcandaceo

Topic  7
yanggang, co, describe, yanggang

Community_7

In [29]:
bag_of_words_7 = tfidf_7.fit_transform(X_7)
lsa_7 = TruncatedSVD(20)
doc_topic_7 = lsa_7.fit_transform(bag_of_words_7)
lsa_7.explained_variance_ratio_

array([0.03919197, 0.0357567 , 0.01990274, 0.01969083, 0.01315743,
       0.01484985, 0.01364197, 0.01366964, 0.01238418, 0.01240378,
       0.01119541, 0.01091663, 0.01062641, 0.00989356, 0.00996021,
       0.00947761, 0.00933236, 0.00866492, 0.00869365, 0.00803585])

In [30]:
display_topics(lsa_7, tfidf_7.get_feature_names(), 10)


Topic  0
usa pageant, sexual comment, comment miss, pageant, pageant shocking, shocking yet, shocking, yet, teen usa, miss teen

Topic  1
pig dictionary, dictionary picture, next male, chauvinist pig, male chauvinist, chauvinist, dictionary, male, picture, pig

Topic  2
still, matt gaetz, gaetz, matt, donald trump, donald, trump, alternative reality, presidential selfie, question moment

Topic  3
donald trump, donald, trump, mcconnell jim, jr mitch, jordan matt, gaetz cruz, graham sean, marco, egg face

Topic  4
boycott, news, fox news, fox, advertiser, invite instruction, represent deep, entitled white, toxic unrest, oiliota

Topic  5
come shit, toxic unrest, instruction, explain entitled, oiliota, entitled white, happy come, infect country, invite instruction, co oiliota

Topic  6
downy, bounty, gillette, dawndish, charmin, dawndish downy, mycascade, crest, pacificlife, cigna

Topic  7
keep, vacation, forced immediate, go forced, keep trend, immediate vacation, vacation keep, forced

Community_8

In [31]:
bag_of_words_8 = tfidf_8.fit_transform(X_8)
lsa_8 = TruncatedSVD(20)
doc_topic_8 = lsa_8.fit_transform(bag_of_words_8)
lsa_8.explained_variance_ratio_

array([0.08073293, 0.03649298, 0.02403689, 0.02419245, 0.01923464,
       0.01737225, 0.01715447, 0.01621799, 0.01586864, 0.01500928,
       0.01415039, 0.01253307, 0.01137147, 0.01092411, 0.01074794,
       0.01016011, 0.00981478, 0.00843264, 0.0081742 , 0.00809893])

In [32]:
display_topics(lsa_8, tfidf_8.get_feature_names(), 10)


Topic  0
idea, mexico, billion, remittance, tax, go nowhere, idea go, discuss idea, sgh, wwlsw sgh

Topic  1
wl, outraged co, vyjbb wl, vyjbb, co vyjbb, medium outraged, outraged, wing medium, left wing, wing

Topic  2
iraqi insult, insult boycott, launch obama, kill iraqi, obama bolton, boycott government, vitriol launch, vitriol, much vitriol, bolton kill

Topic  3
form racism, atlantic, neo liberal, co qrfuizw, zionist neo, actually form, wokezionism, wokezionism co, supremacy lmao, predict

Topic  4
antifa, antifa plan, home, harass, plan, local government, harass whomever, creepy proud, whomever want, plan protest

Topic  5
live, good, revenge screw, yib, hater desperately, tweet hatred, pcxu yib, marm nothing, nothing good, marm

Topic  6
name, fir sex, wanna, fir, ac start, dm name, two play, leak name, date wanna, start leak

Topic  7
wdyknkxroh, co wdyknkxroh, call brian, stelter eunuch, eunuch co, brian stelter, brian, eunuch, stelter, call

Topic  8
resource, dehumanising c

Community_9

In [33]:
bag_of_words_9 = tfidf_9.fit_transform(X_9)
lsa_9 = TruncatedSVD(20)
doc_topic_9 = lsa_9.fit_transform(bag_of_words_9)
lsa_9.explained_variance_ratio_

array([0.20627525, 0.10119706, 0.09643892, 0.02680589, 0.0267337 ,
       0.02730291, 0.01500499, 0.01079307, 0.01022484, 0.00904284,
       0.00863446, 0.00874451, 0.00849691, 0.00747321, 0.0069603 ,
       0.00680894, 0.00682217, 0.00674582, 0.00590569, 0.00580185])

In [34]:
display_topics(lsa_9, tfidf_9.get_feature_names(), 10)


Topic  0
white, clip, create civilization, create, civilization, racist, old audio, dnmvc fpgd, dnmvc, civilization co

Topic  1
wing thru, right wing, interwebs, interwebs really, read, connect, connect ado, account, tweet ann, account prior

Topic  2
retweet nazi, gm america, trump tell, tell fcc, fcc investigate, weekend, misogynist retweet, weekend trump, past weekend, stand racist

Topic  3
comment miss, pageant shocking, shocking, shocking yet, usa pageant, sexual comment, teen usa, usa, miss teen, yet

Topic  4
open, co ozwnxk, ozwnxk, try tell, sucker, ago sucker, sucker firetuckercarlson, tell year, firetuckercarlson co, year ago

Topic  5
try tell, ago sucker, co ozwnxk, ozwnxk, tell year, sucker firetuckercarlson, sucker, firetuckercarlson co, year ago, ago

Topic  6
iraqi, everything white, everything, blame white, man everything, monkey, lunatic, primative, supremacist black, monkey obama

Topic  7
semiliterate, semiliterate primitive, uncover radio, uncover, recently unc

Community_10

In [35]:
bag_of_words_10 = tfidf_10.fit_transform(X_10)
lsa_10 = TruncatedSVD(20)
doc_topic_10 = lsa_10.fit_transform(bag_of_words_10)
lsa_10.explained_variance_ratio_

array([0.03693118, 0.03412557, 0.02451661, 0.02320222, 0.01896729,
       0.01160966, 0.01025512, 0.00962941, 0.00765032, 0.00662608,
       0.00677669, 0.00657745, 0.00637264, 0.00586137, 0.00532975,
       0.00543159, 0.00480619, 0.00468065, 0.00425979, 0.00396333])

In [36]:
display_topics(lsa_10, tfidf_10.get_feature_names(), 10)


Topic  0
please email, call tell, enough angry, advertise foxnew, action please, left silence, silence conservative, com, news advertise, conservative action

Topic  1
mj, jr, ezkanon si, bowcrazyapril, fmeonlylol, ezkanon, marina jr, marina, burnedspy, si

Topic  2
apologize, mob, outrage, never apologize, control never, mob trump, ploy silence, trump show, co ivgt, outrage ploy

Topic  3
deplatform every, serve nothing, continue hunt, nothing indication, every effective, officially throe, voice important, effective conservative, attack judgejeanine, election medium

Topic  4
tpusa, gop marklevinshow, benshapiro tedcruz, donaldjtrumpjr tpusa, marklevinshow foxnew, tpusa charliekirk, tedcruz gop, senategop benshapiro, tedcruz, realdonaldtrump senategop

Topic  5
people, kkk, gold call, people jap, byrd denigrate, paykmznp, leader robert, matter campaign, gold, bag jewish

Topic  6
fake outrage, twqrp yhmv, co twqrp, direct twitter, censor co, twqrp, response fake, twitter actively, ou

Community_11

In [37]:
bag_of_words_11 = tfidf_11.fit_transform(X_11)
lsa_11 = TruncatedSVD(20)
doc_topic_11 = lsa_11.fit_transform(bag_of_words_11)
lsa_11.explained_variance_ratio_

array([0.0507642 , 0.0527984 , 0.02808798, 0.02235737, 0.02095825,
       0.01311214, 0.01304679, 0.0127442 , 0.01177862, 0.01017788,
       0.00811887, 0.00839674, 0.00835898, 0.00814437, 0.00780696,
       0.00757053, 0.00733945, 0.00681884, 0.00659623, 0.00632224])

In [38]:
display_topics(lsa_11, tfidf_11.get_feature_names(), 10)


Topic  0
speech, donaldson, news, kind speech, unchecked change, left unchecked, speech left, unchecked, change country, country forever

Topic  1
co vxfrb, ni googlealert, vxfrb, ni, vxfrb ni, googlealert, course co, koch course, propaganda koch, bankroll hateful

Topic  2
usa pageant, sexual comment, comment miss, pageant shocking, shocking yet, shocking, teen usa, miss teen, usa, pageant

Topic  3
poison body, complicity inject, body politic, search soul, soul complicity, network boss, inject poison, time advertiser, advertiser network, inject

Topic  4
sympathy use, zero sympathy, monkey zero, paper fork, fork, use toilet, zero, toilet paper, sympathy, paper

Topic  5
protester urge, show protester, news show, join co, pirro fox, urge company, company join, ditch pirro, ditch, join

Topic  6
opinion go, hn wxsnkca, wxsnkca, hn, co hn, go co, opinion, go, co, next

Topic  7
pig dictionary, chauvinist pig, male chauvinist, next male, dictionary picture, chauvinist, dictionary, pig, 

Community_12

In [39]:
bag_of_words_12 = tfidf_12.fit_transform(X_12)
lsa_12 = TruncatedSVD(20)
doc_topic_12 = lsa_12.fit_transform(bag_of_words_12)
lsa_12.explained_variance_ratio_

array([0.03869903, 0.04506407, 0.04011948, 0.02975934, 0.02416634,
       0.02254887, 0.02159857, 0.0195082 , 0.01822028, 0.01608382,
       0.0142996 , 0.01210207, 0.01065135, 0.01124858, 0.01021679,
       0.01014777, 0.00908826, 0.00820547, 0.00793885, 0.00739338])

In [40]:
display_topics(lsa_12, tfidf_12.get_feature_names(), 10)


Topic  0
tuckercarlsonfuckshisroomba, gc, co gc, gc gkfwnrq, gkfwnrq, tuckercarlsonfuckshisroomba co, roomba deserve, dj roomba, deserve tuckercarlsonfuckshisroomba, dj

Topic  1
roomba deserve, deserve tuckercarlsonfuckshisroomba, dj roomba, dj, deserve, roomba, tuckercarlsonfuckshisroomba, fuck, refer black, friend tuckercarlsonfuckshisroomba

Topic  2
refer black, yes refer, record yes, friend tuckercarlsonfuckshisroomba, black friend, friend, record, refer, yes, black

Topic  3
mean show, show proof, proof tuckercarlsonfuckshisroomba, proof, mean, show, tuckercarlsonfuckshisroomba, fuck tuckercarlsonfuckshisroomba, fuck, show tuckercarlsonfuckshisroomba

Topic  4
fuck tuckercarlsonfuckshisroomba, fuck, tuckercarlsonfuckshisroomba, fuck roomba, roomba tuckercarlsonfuckshisroomba, roomba, suck, suck tuckercarlsonfuckshisroomba, true fuck, true

Topic  5
still cable, tv show, show tuckercarlsonfuckshisroomba, cable tv, cable, tuckercarlsonfuckshisroomba foxnew, tv, foxnew, still, sho

Now I have my top words in each LSA object w/ 20 topics each. I'm going to pickle each model and then use a new file to:
 - Find which tweets score the highest in each topic to get examples
 - Name communities
 - Sentiment analysis (maybe weekend)

In [41]:
with open('/Users/robertpagano/metis_data/project_5/topic_modeling/lsa_tucker/lsa_0.pickle', 'wb') as to_write:
    pickle.dump(lsa_0, to_write)
    
with open('/Users/robertpagano/metis_data/project_5/topic_modeling/lsa_tucker/lsa_1.pickle', 'wb') as to_write:
    pickle.dump(lsa_1, to_write)
    
with open('/Users/robertpagano/metis_data/project_5/topic_modeling/lsa_tucker/lsa_2.pickle', 'wb') as to_write:
    pickle.dump(lsa_2, to_write)
    
with open('/Users/robertpagano/metis_data/project_5/topic_modeling/lsa_tucker/lsa_3.pickle', 'wb') as to_write:
    pickle.dump(lsa_3, to_write)
    
with open('/Users/robertpagano/metis_data/project_5/topic_modeling/lsa_tucker/lsa_4.pickle', 'wb') as to_write:
    pickle.dump(lsa_4, to_write)
    
with open('/Users/robertpagano/metis_data/project_5/topic_modeling/lsa_tucker/lsa_5.pickle', 'wb') as to_write:
    pickle.dump(lsa_5, to_write)
    
with open('/Users/robertpagano/metis_data/project_5/topic_modeling/lsa_tucker/lsa_6.pickle', 'wb') as to_write:
    pickle.dump(lsa_6, to_write)
    
with open('/Users/robertpagano/metis_data/project_5/topic_modeling/lsa_tucker/lsa_7.pickle', 'wb') as to_write:
    pickle.dump(lsa_7, to_write)
    
with open('/Users/robertpagano/metis_data/project_5/topic_modeling/lsa_tucker/lsa_8.pickle', 'wb') as to_write:
    pickle.dump(lsa_8, to_write)
    
with open('/Users/robertpagano/metis_data/project_5/topic_modeling/lsa_tucker/lsa_9.pickle', 'wb') as to_write:
    pickle.dump(lsa_9, to_write)
    
with open('/Users/robertpagano/metis_data/project_5/topic_modeling/lsa_tucker/lsa_10.pickle', 'wb') as to_write:
    pickle.dump(lsa_10, to_write)
    
with open('/Users/robertpagano/metis_data/project_5/topic_modeling/lsa_tucker/lsa_11.pickle', 'wb') as to_write:
    pickle.dump(lsa_11, to_write)
    
with open('/Users/robertpagano/metis_data/project_5/topic_modeling/lsa_tucker/lsa_12.pickle', 'wb') as to_write:
    pickle.dump(lsa_12, to_write)
      

NMF Models?

In [42]:
nmf_0 = NMF(20)
doc_topic_NMF_0 = nmf_0.fit_transform(bag_of_words_0)
print(nmf_0.reconstruction_err_)

display_topics(nmf_0, tfidf_0.get_feature_names(), 10)

86.50549919623006

Topic  0
co iorzbp, anx, iorzbp anx, iorzbp, exclusive, exclusive sexual, yet unearthed, audio co, unearthed audio, shocking yet

Topic  1
republican defend, shopping mall, ban shopping, moore ban, dressing, dressing room, mall harass, lecture family, mall, brag go

Topic  2
wife, good, vulnerable wound, mean wife, herd probably, wife mean, usa vulnerable, old miss, pretty good, probably pretty

Topic  3
qcozayvyic, co qcozayvyic, exclusive sexual, exclusive, yet unearthed, audio co, unearthed audio, shocking yet, pageant shocking, shocking

Topic  4
rjw, ob rjw, child beauty, repeatedly talk, co ob, talk sex, ob, pageant co, sex child, repeatedly

Topic  5
vile sort, wubucc, predatory disgusting, co wubucc, room language, dtj, defend stop, stop co, language predatory, sort locker

Topic  6
haodcdldqg, co haodcdldqg, bayer still, still advertise, advertise show, show co, advertise, bayer, still, show

Topic  7
fox news, news, fox, protest, protest place, headquarters

In [43]:
nmf_1 = NMF(20)
doc_topic_NMF_1 = nmf_1.fit_transform(bag_of_words_1)
print(nmf_1.reconstruction_err_)

display_topics(nmf_1, tfidf_1.get_feature_names(), 10)

86.50549919379984

Topic  0
co iorzbp, anx, iorzbp anx, iorzbp, exclusive, exclusive sexual, yet unearthed, audio co, unearthed audio, shocking yet

Topic  1
republican defend, shopping mall, ban shopping, moore ban, dressing, dressing room, mall harass, lecture family, mall, brag go

Topic  2
wife, good, vulnerable wound, mean wife, herd probably, wife mean, usa vulnerable, old miss, pretty good, probably pretty

Topic  3
qcozayvyic, co qcozayvyic, exclusive sexual, exclusive, yet unearthed, audio co, unearthed audio, shocking yet, pageant shocking, shocking

Topic  4
rjw, ob rjw, child beauty, repeatedly talk, co ob, talk sex, ob, pageant co, sex child, repeatedly

Topic  5
vile sort, wubucc, predatory disgusting, co wubucc, room language, dtj, defend stop, stop co, language predatory, sort locker

Topic  6
haodcdldqg, co haodcdldqg, bayer still, still advertise, advertise show, show co, advertise, bayer, still, show

Topic  7
fox news, news, fox, protest, protest place, headquarters

In [44]:
nmf_2 = NMF(20)
doc_topic_NMF_2 = nmf_2.fit_transform(bag_of_words_2)
print(nmf_2.reconstruction_err_)

display_topics(nmf_2, tfidf_2.get_feature_names(), 10)

314.11402869516274

Topic  0
every effective, attempt deplatform, effective conservative, hunt attempt, deplatform every, matter continue, officially throe, continue hunt, week serve, nothing indication

Topic  1
call brian, stelter eunuch, co wdyknkxroh, wdyknkxroh, eunuch co, brian stelter, eunuch, stelter, brian, call

Topic  2
ngcc, co ngcc, ngcc nk, nk vt, destroy judgejeanine, nk, vt, judgejeanine co, destroy, judgejeanine

Topic  3
fox news, news, fox, hannity retweet, seem goal, rhetoric hiring, bogguio, co bogguio, brazile cancellation, cancellation judge

Topic  4
people, tran people, leader robert, kkk leader, matter campaign, praise kkk, jap praise, robert byrd, describe bag, campaign write

Topic  5
co xqb, jeanine happen, destroy judge, happen co, xqb, happen, destroy, judge jeanine, judge, jeanine

Topic  6
co xyisuty, xyisuty, xyisuty judgejeanine, foxnew number, number co, judgejeanine salemmediagrp, salemmediagrp, go foxnew, number, go

Topic  7
frat, frat boy, boy jo

In [45]:
nmf_3 = NMF(20)
doc_topic_NMF_3 = nmf_3.fit_transform(bag_of_words_3)
print(nmf_3.reconstruction_err_)

display_topics(nmf_3, tfidf_3.get_feature_names(), 10)

221.9337962406793

Topic  0
sh year, less sh, retweet give, give less, sh, less, boy joke, old frat, frat boy, frat

Topic  1
people, campaign write, lead medium, byrd denigrate, guy lead, leader robert, kkk leader, denigrate, denigrate tran, describe bag

Topic  2
carusone, writing, writing medium, dkz, co dkz, carusone note, earlier week, network earlier, carusone network, semitic bigoted

Topic  3
officially throe, deplatform every, throe election, throe, indication officially, week serve, hunt attempt, nothing indication, continue hunt, serve nothing

Topic  4
minute pure, matter two, savagery move, pure savagery, co apjtgsa, apjtgsa va, savagery, burie medium, move goangelo, apjtgsa

Topic  5
president congrat, mutually, night rule, assured, hbk, matter fire, destruction exactly, warn last, mutually assured, ycp

Topic  6
wdyknkxroh, co wdyknkxroh, stelter eunuch, call brian, eunuch co, brian stelter, brian, eunuch, stelter, call

Topic  7
facebook allow, home facebook, plan haras

In [46]:
nmf_4 = NMF(20)
doc_topic_NMF_4 = nmf_4.fit_transform(bag_of_words_4)
print(nmf_4.reconstruction_err_)

display_topics(nmf_4, tfidf_4.get_feature_names(), 10)

38.14936459958923

Topic  0
icymi white, icymi, co skbkckxj, skbkckxj, supremacist co, white supremacist, supremacist, white, co, bee

Topic  1
trump, dossier, russia, start, reliable co, australia, lie dossier, australia top, diplomat, diplomat uk

Topic  2
unearth different, deeply upsetting, upsetting, upsetting comment, different radio, radio tv, comment recently, recently unearth, vtwi, eblgx

Topic  3
call, ex iqn, jai ex, jai, iqn, co jai, ex, weekend wake, call radio, pain

Topic  4
usa pageant, shocking yet, shocking, pageant shocking, comment miss, pageant, sexual comment, yet, sexual, miss teen

Topic  5
co ov, glamorous mollyjongfast, wdr, glamorous, piece glamorous, fabulous, fabulous piece, ov, ov wdr, mollyjongfast co

Topic  6
co vxfrb, bankroll hateful, hateful propaganda, koch course, googlealert, propaganda koch, vxfrb ni, vxfrb, ni googlealert, ni

Topic  7
piece mollyjongfast, far excellent, want co, co syotaodv, excellent people, show want, syotaodv, favorite piec

In [47]:
nmf_5 = NMF(20)
doc_topic_NMF_5 = nmf_5.fit_transform(bag_of_words_5)
print(nmf_5.reconstruction_err_)

display_topics(nmf_5, tfidf_5.get_feature_names(), 10)

47.27565519532118

Topic  0
suck, loomer everyone, precisely suck, precisely, else suck, suck today, wohl candace, suck precisely, laura loomer, owen laura

Topic  1
vulnerable underage, girl shout, public bathroom, neutral public, fuck vulnerable, bathroom endanger, shout want, endanger girl, endanger, want fuck

Topic  2
support effort, gofundme, maher gofundme, gofundme support, effort, maher, bill maher, bill, support, promise bill

Topic  3
complicit, course, complicit course, shapiro complicit, pewdiepie complicit, course donald, course complicit, owen complicit, course candace, course ben

Topic  4
chan, anyone, trump, vivian, manosphere, tumblr action, tumblr, wing talk, anyone manosphere, avatar

Topic  5
bulrcfl, must something, something right, co bulrcfl, instantly demonetize, demonetize mean, video instantly, demonetize, mean must, instantly

Topic  6
shocking, pageant shocking, shocking yet, sexual comment, usa pageant, comment miss, pageant, yet unearthed, exclusive sexu

In [48]:
nmf_6 = NMF(20)
doc_topic_NMF_6 = nmf_6.fit_transform(bag_of_words_6)
print(nmf_6.reconstruction_err_)

display_topics(nmf_6, tfidf_6.get_feature_names(), 10)

21.032618270744162

Topic  0
blame, williamlegate islam, briannawu candace, online, nt gun, nt, blame williamlegate, blame online, islam blame, blame nt

Topic  1
limitation dumb, statute limitation, different conservative, matter different, stand world, gunn fire, air medium, place fox, news air, tweet statute

Topic  2
retweet give, sh year, sh, give less, less sh, frat, frat boy, boy, boy joke, old frat

Topic  3
old dirt, dirt, dig old, dig, medium matter, matter, medium, old, finally, co tsduhcnzly

Topic  4
use, solar, theboltreport credlin, india use, india, save planet, solar panel, energypoverty, energy intensive, energy

Topic  5
limb, brown, halfway world, skinned year, skinned, co abe, wannabe, wannabe show, show bad, shitty howard

Topic  6
ingrahamangle realcandaceo, realomarnavarro kthopkin, skynew piersmorgan, skynew, judgejeanine dbongino, lindseygrahamsc politicalshort, lisamurkowski, waynedupreeshow cernovich, jeromecorsi realomarnavarro, michaelcohen

Topic  7
yangg

In [49]:
nmf_7 = NMF(20)
doc_topic_NMF_7 = nmf_7.fit_transform(bag_of_words_7)
print(nmf_7.reconstruction_err_)

display_topics(nmf_7, tfidf_7.get_feature_names(), 10)

248.06255196196247

Topic  0
firetuckercarlson boycotttuckercarlson, yet firetuckercarlson, boycotttuckercarlson co, co kuer, kuer, kuer sl, sl, boycotttuckercarlson, usa pageant, sexual comment

Topic  1
dictionary picture, next male, pig dictionary, chauvinist pig, male chauvinist, chauvinist, dictionary, male, picture, pig

Topic  2
still, fox live, congress cindy, question moment, gaetz still, acosta still, presidential selfie, alternative reality, still congress, moment alex

Topic  3
donald trump, donald, trump, graham sean, gaetz cruz, jordan matt, jr mitch, mcconnell jim, marco rubio, rubio lindsey

Topic  4
unrest infect, instruction, oiliota, infect country, entitled white, explain entitled, come shit, supremacist represent, dm invite, happy come

Topic  5
hotelsdotcom, travelocity, hotelsdotcom orbitz, orbitz, expedia travelocity, expediagroup, expedia, orbitz boycott, expediagroup expedia, allow bigotry

Topic  6
downy, bounty, gillette, dawndish, charmin, dawndish downy, m

In [50]:
nmf_8 = NMF(20)
doc_topic_NMF_8 = nmf_8.fit_transform(bag_of_words_8)
print(nmf_8.reconstruction_err_)

display_topics(nmf_8, tfidf_8.get_feature_names(), 10)

52.34866073781815

Topic  0
idea, mexico, billion, remittance, tax, go nowhere, tax billion, tax lobbyist, sgh, wwlsw sgh

Topic  1
outraged co, wl, vyjbb wl, vyjbb, co vyjbb, medium outraged, outraged, wing medium, left wing, wing

Topic  2
obama bolton, iraqi insult, kill iraqi, vitriol launch, much vitriol, vitriol, government boycotttuckercarlson, boycott government, insult boycott, launch obama

Topic  3
zionist neo, co qrfuizw, magazine, liberal magazine, predict, lmao wokezionism, qrfuizw, thisth zionist, magazine atlantic, neo liberal

Topic  4
antifa, harass whomever, government protection, het, co het, protection harass, friday night, antifa thug, pm creepy, want co

Topic  5
live, good, pcxu, unhappy, unhappy school, revenge screw, live long, hater desperately, hatred co, productive good

Topic  6
name, sex gay, name college, wanna, hn, wanna go, fir sex, start leak, hn hley, fir

Topic  7
co wdyknkxroh, wdyknkxroh, call brian, stelter eunuch, eunuch co, brian stelter, brian

In [51]:
nmf_9 = NMF(20)
doc_topic_NMF_9 = nmf_9.fit_transform(bag_of_words_9)
print(nmf_9.reconstruction_err_)

display_topics(nmf_9, tfidf_9.get_feature_names(), 10)

15.253586273390718

Topic  0
white, clip, create civilization, create, civilization, racist, civilization hear, duh, reveal, fpgd

Topic  1
detective, coulter smoking, interwebs, connect, connect ado, wing, account, account prior, wing thru, fake twitter

Topic  2
foxnew reinstate, nbcsnl, deceased senjohnmccain, retweet nazi, nazi jackposobiec, jackposobiec, gm america, deny white, jackposobiec deny, judgejeanine gm

Topic  3
unearthed audio, exclusive sexual, exclusive, yet unearthed, unearthed, audio co, pageant shocking, shocking, shocking yet, usa pageant

Topic  4
open, many disagree, open national, discussion many, wdyq uwjjw, rashadrichey politicalanalyst, uwjjw, rashadrichey, also open, hit

Topic  5
sucker firetuckercarlson, ozwnxk, sucker, tell year, ago sucker, try tell, co ozwnxk, firetuckercarlson co, year ago, ago

Topic  6
white, everything white, everything, blame white, man everything, jk, obama, iraqi primative, sister muslim, primative monkey

Topic  7
iraqi semilit

In [52]:
nmf_10 = NMF(20)
doc_topic_NMF_10 = nmf_10.fit_transform(bag_of_words_10)
print(nmf_10.reconstruction_err_)

display_topics(nmf_10, tfidf_10.get_feature_names(), 10)

52.1399451193262

Topic  0
advertise foxnew, news advertise, action please, angry left, com, silence conservative, foxnew com, please email, call tell, enough angry

Topic  1
mj, jr, si bowcrazyapril, tina, ezkanon si, bowcrazyapril fmeonlylol, bowcrazyapril, ezkanon, si, marina jr

Topic  2
apologize, mob, outrage, never apologize, fake moral, mob trump, bow liberal, ploy silence, co ivgt, medium control

Topic  3
attack judgejeanine, deplatform every, election medium, every effective, throe, serve nothing, voice important, throe election, indication, indication officially

Topic  4
senategop benshapiro, donaldjtrumpjr tpusa, gop marklevinshow, marklevinshow foxnew, tpusa, benshapiro tedcruz, tpusa charliekirk, tedcruz gop, tedcruz, realdonaldtrump senategop

Topic  5
people, co paykmznp, people jap, radio comment, paykmznp, byrd denigrate, campaign write, describe bag, gold, gold call

Topic  6
twqrp, actively try, outrage direct, yhmv, try censor, response fake, direct twitter, co t

In [53]:
nmf_11 = NMF(20)
doc_topic_NMF_11 = nmf_11.fit_transform(bag_of_words_11)
print(nmf_11.reconstruction_err_)

display_topics(nmf_11, tfidf_11.get_feature_names(), 10)

162.33400445757422

Topic  0
speech, news, zk uoifw, ttfj, ttfj oi, uoifw co, uoifw, rl ttfj, co rl, oi

Topic  1
vxfrb, co vxfrb, ni, vxfrb ni, ni googlealert, googlealert, course co, koch course, propaganda koch, bankroll hateful

Topic  2
yet unearthed, exclusive sexual, exclusive, unearthed audio, unearthed, audio co, pageant shocking, shocking yet, shocking, usa pageant

Topic  3
poison body, search soul, inject poison, complicity inject, body politic, network boss, soul complicity, time advertiser, advertiser network, inject

Topic  4
sympathy use, zero sympathy, monkey zero, paper fork, fork, use toilet, zero, sympathy, toilet paper, paper

Topic  5
protester urge, show protester, join co, pirro fox, urge company, company join, ditch pirro, join, urge, ditch

Topic  6
opinion go, hn, wxsnkca, co hn, hn wxsnkca, go co, opinion, go, co, week

Topic  7
male chauvinist, pig dictionary, chauvinist pig, next male, chauvinist, dictionary picture, dictionary, pig, male, picture

Topic  

In [54]:
nmf_12 = NMF(20)
doc_topic_NMF_12 = nmf_12.fit_transform(bag_of_words_12)
print(nmf_12.reconstruction_err_)

display_topics(nmf_12, tfidf_12.get_feature_names(), 10)

31.60961042884443

Topic  0
co gc, gc gkfwnrq, gc, gkfwnrq, tuckercarlsonfuckshisroomba co, co, tuckercarlsonfuckshisroomba, asshat, asshat tuckercarlsonfuckshisroomba, face co

Topic  1
roomba deserve, dj, deserve tuckercarlsonfuckshisroomba, dj roomba, deserve, roomba, tuckercarlsonfuckshisroomba, deserve good, old, good

Topic  2
refer black, yes refer, friend tuckercarlsonfuckshisroomba, record yes, black friend, friend, record, refer, yes, black

Topic  3
proof tuckercarlsonfuckshisroomba, mean show, show proof, proof, mean, show, tuckercarlsonfuckshisroomba, asshat tuckercarlsonfuckshisroomba, asshat, mean maybe

Topic  4
fuck tuckercarlsonfuckshisroomba, fuck, tuckercarlsonfuckshisroomba, asshat tuckercarlsonfuckshisroomba, asshat, tuckercarlsonfuckshisroomba tuckercarlsonfuckshisroomba, ha, ha ha, prove, real lie

Topic  5
tv show, cable tv, still cable, show tuckercarlsonfuckshisroomba, cable, tuckercarlsonfuckshisroomba foxnew, tv, still, foxnew, show

Topic  6
stick penis, a

In [55]:
with open('/Users/robertpagano/metis_data/project_5/topic_modeling/nmf_tucker/nmf_0.pickle', 'wb') as to_write:
    pickle.dump(nmf_0, to_write)
    
with open('/Users/robertpagano/metis_data/project_5/topic_modeling/nmf_tucker/nmf_1.pickle', 'wb') as to_write:
    pickle.dump(nmf_1, to_write)
    
with open('/Users/robertpagano/metis_data/project_5/topic_modeling/nmf_tucker/nmf_2.pickle', 'wb') as to_write:
    pickle.dump(nmf_2, to_write)
    
with open('/Users/robertpagano/metis_data/project_5/topic_modeling/nmf_tucker/nmf_3.pickle', 'wb') as to_write:
    pickle.dump(nmf_3, to_write)
    
with open('/Users/robertpagano/metis_data/project_5/topic_modeling/nmf_tucker/nmf_4.pickle', 'wb') as to_write:
    pickle.dump(nmf_4, to_write)
    
with open('/Users/robertpagano/metis_data/project_5/topic_modeling/nmf_tucker/nmf_5.pickle', 'wb') as to_write:
    pickle.dump(nmf_5, to_write)
    
with open('/Users/robertpagano/metis_data/project_5/topic_modeling/nmf_tucker/nmf_6.pickle', 'wb') as to_write:
    pickle.dump(nmf_6, to_write)
    
with open('/Users/robertpagano/metis_data/project_5/topic_modeling/nmf_tucker/nmf_7.pickle', 'wb') as to_write:
    pickle.dump(nmf_7, to_write)
    
with open('/Users/robertpagano/metis_data/project_5/topic_modeling/nmf_tucker/nmf_8.pickle', 'wb') as to_write:
    pickle.dump(nmf_8, to_write)
    
with open('/Users/robertpagano/metis_data/project_5/topic_modeling/nmf_tucker/nmf_9.pickle', 'wb') as to_write:
    pickle.dump(nmf_9, to_write)
    
with open('/Users/robertpagano/metis_data/project_5/topic_modeling/nmf_tucker/nmf_10.pickle', 'wb') as to_write:
    pickle.dump(nmf_10, to_write)
    
with open('/Users/robertpagano/metis_data/project_5/topic_modeling/nmf_tucker/nmf_11.pickle', 'wb') as to_write:
    pickle.dump(nmf_11, to_write)
    
with open('/Users/robertpagano/metis_data/project_5/topic_modeling/nmf_tucker/nmf_12.pickle', 'wb') as to_write:
    pickle.dump(nmf_12, to_write)


    

Running Topic Modeling on ALL text data, both NMF and LSA, on various topic counts, to feed into DBScan / KMeans for cluster analysis

In [146]:
# ## Creating list of the processed text files to merge, so it's all in the same format and can re-use code
# dfs_to_merge = [df_0, df_1, df_2, df_3, df_4, df_5, df_6, df_7, df_8, df_9, df_10, df_11, df_26, df_40]

In [149]:
# total = 0

# for df in dfs_to_merge:
#     print(df.shape)
#     total += df.shape[0]
    
# print(total)

(34997, 10)
(1446, 10)
(8345, 10)
(9346, 10)
(224793, 10)
(237746, 10)
(190998, 10)
(135612, 10)
(8192, 10)
(425343, 10)
(13157, 10)
(2270, 10)
(1141, 10)
(2628, 10)
1296014


In [150]:
# df_all_preprocessed = pd.concat(dfs_to_merge)

In [151]:
# df_all_preprocessed.shape

(1296014, 10)

In [153]:
# df_all = df_all_preprocessed.copy()

In [157]:
# df_all.head()

screen_name  followers_count  modularity_class  \
405812   DavidJHarrisJr            75631               0.0   
712255   DavidJHarrisJr            75652               0.0   
465115   DavidJHarrisJr            75638               0.0   
62994    DavidJHarrisJr            75620               0.0   
1205837  DavidJHarrisJr            75690               0.0   

                                                                                                                                              main_text  \
405812   So @aoc answered a casting call... and now she’s in Congress! @RealJamesWoods and @realDonaldTrump have to see this!!! https://t.co/HBUW78XqAo   
712255                                                       So @aoc wants to raise our taxes and she hasn’t even paid her own! https://t.co/W1m1CCnN3Y   
465115                                                            Now @aoc says America is garbage? \nWho elected this woman!!! https://t.co/O7hsaAK9jH   
62994              Former FEC Commissioner says there’s more than enough evidence to launch a criminal investigation into @aoc! https://t.co/XPL6LhrZFD   
1205837                                                    Hey AOC! Guess what? Climate change and global warming IS a hoax!!!! https://t.co/OwY68EMVMm   

        retweet_text                      tweet_date  PageRank  rt_count  \
405812           NaN  Mon Mar 11 01:16:04 +0000 2019  0.004242      6936   
712255           NaN  Sun Mar 10 03:49:44 +0000 2019  0.004242      1127   
465115           NaN  Sun Mar 10 22:30:39 +0000 2019  0.004242      1056   
62994            NaN  Tue Mar 12 01:25:16 +0000 2019  0.004242       997   
1205837          NaN  Fri Mar 08 05:02:07 +0000 2019  0.004242       838   

                                                                                                                                             final_text  \
405812   So @aoc answered a casting call... and now she’s in Congress! @RealJamesWoods and @realDonaldTrump have to see this!!! https://t.co/HBUW78XqAo   
712255                                                       So @aoc wants to raise our taxes and she hasn’t even paid her own! https://t.co/W1m1CCnN3Y   
465115                                                            Now @aoc says America is garbage? \nWho elected this woman!!! https://t.co/O7hsaAK9jH   
62994              Former FEC Commissioner says there’s more than enough evidence to launch a criminal investigation into @aoc! https://t.co/XPL6LhrZFD   
1205837                                                    Hey AOC! Guess what? Climate change and global warming IS a hoax!!!! https://t.co/OwY68EMVMm   

                                                                                                                         tweet_processed  
405812    so aoc answer a cast call and now -PRON- s in congress realjameswood and realdonaldtrump have to see this https t co hbuw xqao  
712255                                       so aoc want to raise -PRON- tax and -PRON- hasn t even pay -PRON- own https t co w m ccnn y  
465115                                                         now aoc say america be garbage who elect this woman https t co o hsaak jh  
62994    former fec commissioner say there s more than enough evidence to launch a criminal investigation into aoc https t co xpl lhrzfd  
1205837                                              hey aoc guess what climate change and global warming be a hoax https t co owy emvmm

In [165]:
# with open('/Users/robertpagano/metis_data/project_5/community_pickles_pre_nlp/preprocessed_dfs/df_all.pickle', 'wb') as to_write:
#     pickle.dump(df_all, to_write)

In [2]:
# with open('/Users/robertpagano/metis_data/project_5/community_pickles_pre_nlp/preprocessed_dfs/df_all.pickle', 'rb') as f:
#     df_all = pickle.load(f)

So the concat worked, the # of rows equals the number of all the rows added up (just to be safe)

Now I'll run a few different LSAs and NMFs on the total data, and move on to clustering

# Preprocess Text - for both LSA and NMF

In [3]:
# # Can come back to tweek this if I notice anything
# tweet_stopwords = stopwords.words('english') + \
#     ['rt', 'https', 'http', 'amp', 'via', 'one', 'around', 'would', 'let', 'could', 'going', 'like', 
#      'get', 'may', 'says', 'say', 'make', 'based', 'even', 'another', 'completely', 'thanks', 'way', 
#      'find', 'used', 'thing', '2019', 'see', 'need', 'know', 'knows', 'think', 'thinks', 'take', 'new', 
#      'day', 'days', 'aoc', 'alexandria', 'ocasio', 'cortez', 'ocasio-cortez', 'pron']

In [4]:
# tfidf_all = TfidfVectorizer(stop_words=tweet_stopwords, token_pattern="\\b[a-z][a-z]+\\b", ngram_range=(1,2))

In [5]:
# X_all = df_all['tweet_processed']

In [6]:
# bag_of_words_all = tfidf_all.fit_transform(X_all)

In [7]:
# type(bag_of_words_all)

scipy.sparse.csr.csr_matrix

# LSA on all text
*10, 15, 20, 25 topics*

In [170]:
# # LSA on 10 topics
# lsa_all_10 = TruncatedSVD(10)
# lsa_all_10.fit_transform(bag_of_words_all)
# lsa_all_10.explained_variance_ratio_

array([0.01284424, 0.01030946, 0.00990273, 0.00920476, 0.00827643,
       0.00781187, 0.00793363, 0.0072784 , 0.00735234, 0.00721214])

In [175]:
# display_topics(lsa_all_10, tfidf_all.get_feature_names(), 10)


Topic  0
claim, box fraud, visit film, represent neighbor, crew postal, film news, never boyfriend, claim visit, apartment local, claim mail

Topic  1
skill libs, libs nice, cookie teach, co cvfc, kjzum, cvfc, cvfc kjzum, boycott cookie, nice job, teach little

Topic  2
file bogus, case conspiracy, allege untrue, spamm file, group spamm, untrue scandal, theory run, scandal misinformation, complaint fox, report allege

Topic  3
society job, live society, job leave, leave die, automate work, automate, die core, core problem, excited live, reason excited

Topic  4
hawaii, crazy, green deal, green, deal, hawaii anyone, crazy green, oppose green, senator mazie, hirono hawaii

Topic  5
address, climate change, inequality address, divide address, change together, together work, address income, tsbac, co tsbac, work sxsw

Topic  6
moderate, time rewrite, edge solve, visionary tinker, naive visionary, moderate naive, moderate spot, problem democracy, democracy economy, critique moderate

Topic

In [176]:
# with open('/Users/robertpagano/metis_data/project_5/topic_modeling/lsa_aoc/lsa_all_10.pickle', 'wb') as to_write:
#     pickle.dump(lsa_all_10, to_write)

---------

In [187]:
# # LSA on 15 topics
# lsa_all_15 = TruncatedSVD(15)
# lsa_all_15.fit_transform(bag_of_words_all)
# lsa_all_15.explained_variance_ratio_

array([0.01284409, 0.01030946, 0.00990181, 0.00920577, 0.00827817,
       0.00780601, 0.00793854, 0.00728814, 0.00735668, 0.00721322,
       0.00614608, 0.00581037, 0.00529221, 0.00522468, 0.00525943])

In [188]:
# display_topics(lsa_all_15, tfidf_all.get_feature_names(), 10)


Topic  0
claim, claim visit, visit film, crew postal, film news, represent neighbor, apartment local, never boyfriend, box fraud, claim mail

Topic  1
skill libs, libs nice, cookie teach, cvfc kjzum, cvfc, co cvfc, kjzum, boycott cookie, nice job, teach little

Topic  2
file bogus, case conspiracy, allege untrue, spamm file, group spamm, untrue scandal, theory run, scandal misinformation, complaint fox, report allege

Topic  3
society job, live society, job leave, leave die, automate work, automate, die core, core problem, excited live, reason excited

Topic  4
hawaii, crazy, green deal, green, deal, hawaii anyone, crazy green, oppose green, senator mazie, hirono hawaii

Topic  5
address, climate change, inequality address, divide address, change together, together work, address income, co tsbac, tsbac, work sxsw

Topic  6
moderate, time rewrite, visionary tinker, edge solve, naive visionary, moderate naive, moderate spot, problem democracy, democracy economy, critique moderate

Topic

In [189]:
# with open('/Users/robertpagano/metis_data/project_5/topic_modeling/lsa_aoc/lsa_all_15.pickle', 'wb') as to_write:
#     pickle.dump(lsa_all_15, to_write)

---------

In [180]:
# # LSA on 20 topics
# lsa_all_20 = TruncatedSVD(20)
# lsa_all_20.fit_transform(bag_of_words_all)
# lsa_all_20.explained_variance_ratio_

array([0.01284416, 0.01031006, 0.00990201, 0.0092044 , 0.00827566,
       0.00780765, 0.00793947, 0.00729166, 0.00735414, 0.00721279,
       0.00615355, 0.00581353, 0.00530054, 0.0052364 , 0.0052897 ,
       0.00525525, 0.00489424, 0.00446149, 0.00414687, 0.00423378])

In [181]:
# display_topics(lsa_all_20, tfidf_all.get_feature_names(), 10)


Topic  0
claim, film news, box fraud, crew postal, visit film, claim visit, apartment local, represent neighbor, never boyfriend, claim mail

Topic  1
skill libs, libs nice, cookie teach, cvfc kjzum, co cvfc, cvfc, kjzum, boycott cookie, nice job, teach little

Topic  2
file bogus, case conspiracy, allege untrue, spamm file, group spamm, untrue scandal, theory run, scandal misinformation, complaint fox, report allege

Topic  3
society job, live society, job leave, leave die, automate work, automate, die core, core problem, excited live, reason excited

Topic  4
hawaii, crazy, green deal, green, deal, hawaii anyone, crazy green, oppose green, senator mazie, hirono hawaii

Topic  5
address, climate change, inequality address, divide address, change together, together work, address income, co tsbac, tsbac, work sxsw

Topic  6
moderate, time rewrite, edge solve, visionary tinker, naive visionary, moderate naive, moderate spot, problem democracy, democracy economy, critique moderate

Topic

In [182]:
# with open('/Users/robertpagano/metis_data/project_5/topic_modeling/lsa_aoc/lsa_all_20.pickle', 'wb') as to_write:
#     pickle.dump(lsa_all_20, to_write)

---------

In [8]:
# # LSA on 25 topics
# lsa_all_25 = TruncatedSVD(25)
# lsa_all_25.fit_transform(bag_of_words_all)
# lsa_all_25.explained_variance_ratio_

array([0.01284428, 0.01031   , 0.00990254, 0.0092052 , 0.0082777 ,
       0.00780795, 0.00793927, 0.00729185, 0.00735446, 0.00721273,
       0.00615261, 0.00580615, 0.00530464, 0.00523566, 0.00529585,
       0.00525649, 0.00492341, 0.0044789 , 0.00419832, 0.00428453,
       0.00406109, 0.00396976, 0.00393049, 0.00365576, 0.00348358])

In [11]:
# display_topics(lsa_all_25, tfidf_all.get_feature_names(), 10)


Topic  0
claim, box fraud, apartment local, represent neighbor, crew postal, never boyfriend, claim visit, film news, visit film, claim mail

Topic  1
skill libs, libs nice, cookie teach, co cvfc, kjzum, cvfc, cvfc kjzum, boycott cookie, nice job, teach little

Topic  2
file bogus, case conspiracy, allege untrue, spamm file, group spamm, untrue scandal, theory run, scandal misinformation, complaint fox, report allege

Topic  3
society job, live society, job leave, leave die, automate work, automate, die core, core problem, excited live, reason excited

Topic  4
hawaii, crazy, green deal, green, deal, hawaii anyone, crazy green, oppose green, senator mazie, hirono hawaii

Topic  5
address, climate change, inequality address, divide address, change together, together work, address income, co tsbac, tsbac, work sxsw

Topic  6
moderate, time rewrite, visionary tinker, edge solve, naive visionary, moderate naive, moderate spot, problem democracy, democracy economy, critique moderate

Topic

In [12]:
# with open('/Users/robertpagano/metis_data/project_5/topic_modeling/lsa_aoc/lsa_all_25.pickle', 'wb') as to_write:
#     pickle.dump(lsa_all_25, to_write)

# NMF on all text
*10, 15, 20, 25 topics*

In [13]:
# # NMF on 10 topics
# nmf_all_10 = NMF(10)
# nmf_all_10.fit_transform(bag_of_words_all)
# print(nmf_all_10.reconstruction_err_)

1085.5968911147454


In [14]:
# display_topics(nmf_all_10, tfidf_all.get_feature_names(), 10)


Topic  0
claim, crew postal, film news, visit film, represent neighbor, claim visit, apartment local, box fraud, never boyfriend, claim mail

Topic  1
skill libs, libs nice, cookie teach, cvfc, cvfc kjzum, kjzum, co cvfc, boycott cookie, nice job, teach little

Topic  2
file bogus, case conspiracy, allege untrue, spamm file, group spamm, untrue scandal, theory run, news report, complaint fox, scandal misinformation

Topic  3
society job, live society, job leave, leave die, automate work, automate, die core, core problem, excited live, reason excited

Topic  4
hawaii, crazy, green deal, green, deal, hawaii anyone, crazy green, oppose green, senator mazie, hirono hawaii

Topic  5
address, inequality address, divide address, change together, together work, address income, co tsbac, tsbac, work sxsw, fear divide

Topic  6
moderate, time rewrite, visionary tinker, edge solve, naive visionary, moderate naive, moderate spot, problem democracy, democracy economy, critique moderate

Topic  7
i

In [15]:
# with open('/Users/robertpagano/metis_data/project_5/topic_modeling/nmf_aoc/nmf_all_10.pickle', 'wb') as to_write:
#     pickle.dump(nmf_all_10, to_write)

---------

In [16]:
# # NMF on 15 topics
# nmf_all_15 = NMF(15)
# nmf_all_15.fit_transform(bag_of_words_all)
# print(nmf_all_15.reconstruction_err_)

1068.5427287389832


In [17]:
# display_topics(nmf_all_15, tfidf_all.get_feature_names(), 10)


Topic  0
claim, claim visit, crew postal, apartment local, box fraud, visit film, represent neighbor, film news, never boyfriend, claim mail

Topic  1
skill libs, libs nice, cookie teach, kjzum, co cvfc, cvfc kjzum, cvfc, boycott cookie, nice job, teach little

Topic  2
file bogus, case conspiracy, allege untrue, spamm file, group spamm, untrue scandal, theory run, news report, scandal misinformation, complaint fox

Topic  3
society job, live society, job leave, leave die, automate work, automate, die core, core problem, excited live, reason excited

Topic  4
hawaii, crazy, green deal, green, deal, hawaii anyone, crazy green, oppose green, senator mazie, hirono hawaii

Topic  5
address, inequality address, divide address, change together, together work, address income, co tsbac, tsbac, work sxsw, fear divide

Topic  6
moderate, time rewrite, visionary tinker, edge solve, naive visionary, moderate naive, moderate spot, problem democracy, democracy economy, critique moderate

Topic  7
i

In [18]:
# with open('/Users/robertpagano/metis_data/project_5/topic_modeling/nmf_aoc/nmf_all_15.pickle', 'wb') as to_write:
#     pickle.dump(nmf_all_15, to_write)

---------

In [19]:
# # NMF on 20 topics
# nmf_all_20 = NMF(20)
# nmf_all_20.fit_transform(bag_of_words_all)
# print(nmf_all_20.reconstruction_err_)

1054.2875702490992


In [20]:
# display_topics(nmf_all_20, tfidf_all.get_feature_names(), 10)


Topic  0
claim, film news, apartment local, crew postal, never boyfriend, box fraud, represent neighbor, visit film, claim visit, claim mail

Topic  1
skill libs, libs nice, cookie teach, co cvfc, kjzum, cvfc kjzum, cvfc, boycott cookie, nice job, teach little

Topic  2
file bogus, case conspiracy, allege untrue, spamm file, group spamm, untrue scandal, theory run, news report, scandal misinformation, complaint fox

Topic  3
society job, live society, job leave, leave die, automate work, automate, die core, core problem, excited live, reason excited

Topic  4
hawaii, crazy, green deal, green, deal, hawaii anyone, crazy green, oppose green, senator mazie, hirono hawaii

Topic  5
address, inequality address, divide address, change together, together work, address income, tsbac, co tsbac, work sxsw, fear divide

Topic  6
moderate, time rewrite, visionary tinker, edge solve, naive visionary, moderate naive, moderate spot, problem democracy, democracy economy, critique moderate

Topic  7
i

In [21]:
# with open('/Users/robertpagano/metis_data/project_5/topic_modeling/nmf_aoc/nmf_all_20.pickle', 'wb') as to_write:
#     pickle.dump(nmf_all_20, to_write)

---------

In [22]:
# # NMF on 25 topics
# nmf_all_25 = NMF(25)
# nmf_all_25.fit_transform(bag_of_words_all)
# print(nmf_all_25.reconstruction_err_)

# ## setting a variable to the fit transformed

1042.4274847932734


In [28]:
# nmf_model_25_df = pd.DataFrame(data=nmf_all_25.fit_transform(bag_of_words_all))
# nmf_model_25_df.shape

(1296014, 25)

In [29]:
# print(nmf_all_25.reconstruction_err_)
# nmf_model_25_df.head()

1042.4277109200966


0         1         2         3    4         5         6         7   \
0  0.0  0.000000  0.000000  0.000000  0.0  0.000000  0.000000  0.000000   
1  0.0  0.000341  0.000030  0.000488  0.0  0.000410  0.000302  0.000239   
2  0.0  0.000201  0.000123  0.000141  0.0  0.000175  0.000109  0.000000   
3  0.0  0.000227  0.000000  0.000168  0.0  0.000199  0.000243  0.002161   
4  0.0  0.000223  0.000073  0.000072  0.0  0.007420  0.000188  0.000149   

         8         9  ...         15        16        17        18        19  \
0  0.000000  0.000000 ...   0.000000  0.112873  0.000000  0.000000  0.000000   
1  0.000329  0.000000 ...   0.000617  0.000773  0.000503  0.000775  0.000000   
2  0.000000  0.010703 ...   0.000017  0.000000  0.000183  0.000109  0.000043   
3  0.000307  0.000000 ...   0.000441  0.000516  0.000245  0.000574  0.000057   
4  0.000092  0.000000 ...   0.000427  0.000664  0.000307  0.000473  0.000000   

         20        21        22        23   24  
0  0.000000  0.000000  0.000000  0.000000  0.0  
1  0.000674  0.003253  0.010097  0.000656  0.0  
2  0.000000  0.015331  0.000000  0.010695  0.0  
3  0.000031  0.000271  0.003239  0.000439  0.0  
4  0.000000  0.000351  0.000020  0.000372  0.0  

[5 rows x 25 columns]

NOTE TO SELF - if the above works, I'll need to make DFs of all versions again (which will take around another hour) :(

Then I'll concat them with index and screen name, so I can assign values back to the tweeters otherwise. One annoying thing is I didn't pass in tweet ID so I'll have to think about that too

In [23]:
# display_topics(nmf_all_25, tfidf_all.get_feature_names(), 10)


Topic  0
claim, apartment local, crew postal, never boyfriend, claim visit, film news, visit film, box fraud, represent neighbor, claim mail

Topic  1
skill libs, libs nice, cookie teach, cvfc, cvfc kjzum, co cvfc, kjzum, boycott cookie, nice job, teach little

Topic  2
file bogus, case conspiracy, allege untrue, spamm file, group spamm, untrue scandal, theory run, news report, scandal misinformation, complaint fox

Topic  3
society job, live society, job leave, leave die, automate work, automate, die core, core problem, excited live, reason excited

Topic  4
hawaii, crazy, green deal, green, deal, hawaii anyone, oppose green, crazy green, senator mazie, hirono hawaii

Topic  5
address, inequality address, divide address, change together, together work, address income, tsbac, co tsbac, work sxsw, fear divide

Topic  6
moderate, time rewrite, edge solve, visionary tinker, naive visionary, moderate naive, moderate spot, problem democracy, democracy economy, critique moderate

Topic  7
i

In [24]:
# with open('/Users/robertpagano/metis_data/project_5/topic_modeling/nmf_aoc/nmf_all_25.pickle', 'wb') as to_write:
#     pickle.dump(nmf_all_25, to_write)

Now I will create DF's where I append this info to a dataframe

then try dbscan with these

In [31]:
# df_topic_base = df_all.reset_index()
# df_topic_base.head()

index     screen_name  followers_count  modularity_class  \
0   405812  DavidJHarrisJr            75631               0.0   
1   712255  DavidJHarrisJr            75652               0.0   
2   465115  DavidJHarrisJr            75638               0.0   
3    62994  DavidJHarrisJr            75620               0.0   
4  1205837  DavidJHarrisJr            75690               0.0   

                                           main_text retweet_text  \
0  So @aoc answered a casting call... and now she...          NaN   
1  So @aoc wants to raise our taxes and she hasn’...          NaN   
2  Now @aoc says America is garbage? \nWho electe...          NaN   
3  Former FEC Commissioner says there’s more than...          NaN   
4  Hey AOC! Guess what? Climate change and global...          NaN   

                       tweet_date  PageRank  rt_count  \
0  Mon Mar 11 01:16:04 +0000 2019  0.004242      6936   
1  Sun Mar 10 03:49:44 +0000 2019  0.004242      1127   
2  Sun Mar 10 22:30:39 +0000 2019  0.004242      1056   
3  Tue Mar 12 01:25:16 +0000 2019  0.004242       997   
4  Fri Mar 08 05:02:07 +0000 2019  0.004242       838   

                                          final_text  \
0  So @aoc answered a casting call... and now she...   
1  So @aoc wants to raise our taxes and she hasn’...   
2  Now @aoc says America is garbage? \nWho electe...   
3  Former FEC Commissioner says there’s more than...   
4  Hey AOC! Guess what? Climate change and global...   

                                     tweet_processed  
0  so aoc answer a cast call and now -PRON- s in ...  
1  so aoc want to raise -PRON- tax and -PRON- has...  
2  now aoc say america be garbage who elect this ...  
3  former fec commissioner say there s more than ...  
4  hey aoc guess what climate change and global w...

crap - now I know why I saved the fit_transform(bag_of_words_ objects previously, that's what I need to use to create the df. For now I'm seeing if I can just call that as the "data" and see if that works. Otherwise I will need to set them all equal to variables

In [32]:
# df_topic_base.shape

(1296014, 11)

In [34]:
# lsa_all_10 = TruncatedSVD(10)

# lsa_model_10_df = pd.DataFrame(data=lsa_all_10.fit_transform(bag_of_words_all))
# lsa_model_10_df.shape

(1296014, 10)

In [36]:
# lsa_all_15 = TruncatedSVD(15)

# lsa_model_15_df = pd.DataFrame(data=lsa_all_15.fit_transform(bag_of_words_all))
# lsa_model_15_df.shape

(1296014, 15)

In [37]:
# lsa_all_20 = TruncatedSVD(20)

# lsa_model_20_df = pd.DataFrame(data=lsa_all_20.fit_transform(bag_of_words_all))
# lsa_model_20_df.shape

(1296014, 20)

In [38]:
# lsa_all_25 = TruncatedSVD(25)

# lsa_model_25_df = pd.DataFrame(data=lsa_all_25.fit_transform(bag_of_words_all))
# lsa_model_25_df.shape

(1296014, 25)

In [39]:
# df_lsa_model_10 = pd.merge(df_topic_base, lsa_model_10_df, left_index=True, right_index=True)
# df_lsa_model_15 = pd.merge(df_topic_base, lsa_model_15_df, left_index=True, right_index=True)
# df_lsa_model_20 = pd.merge(df_topic_base, lsa_model_20_df, left_index=True, right_index=True)
# df_lsa_model_25 = pd.merge(df_topic_base, lsa_model_25_df, left_index=True, right_index=True)

---------

In [40]:
# nmf_all_10 = NMF(10)

# nmf_model_10_df = pd.DataFrame(data=nmf_all_10.fit_transform(bag_of_words_all))
# nmf_model_10_df.shape

(1296014, 10)

In [41]:
# nmf_all_15 = NMF(15)

# nmf_model_15_df = pd.DataFrame(data=nmf_all_15.fit_transform(bag_of_words_all))
# nmf_model_15_df.shape

(1296014, 15)

In [42]:
# nmf_all_20 = NMF(20)

# nmf_model_20_df = pd.DataFrame(data=nmf_all_20.fit_transform(bag_of_words_all))
# nmf_model_20_df.shape

(1296014, 20)

In [ ]:
# # nmf_model_25_df = pd.DataFrame(data=nmf_all_25.fit_transform(bag_of_words_all)) ALREADY DONE!
# nmf_model_25_df.shape

In [43]:
# df_nmf_model_10 = pd.merge(df_topic_base, nmf_model_10_df, left_index=True, right_index=True)
# df_nmf_model_15 = pd.merge(df_topic_base, nmf_model_15_df, left_index=True, right_index=True)
# df_nmf_model_20 = pd.merge(df_topic_base, nmf_model_20_df, left_index=True, right_index=True)
# df_nmf_model_25 = pd.merge(df_topic_base, nmf_model_25_df, left_index=True, right_index=True)


In [44]:
# with open('/Users/robertpagano/metis_data/project_5/topic_modeling/topic_dfs/df_lsa_model_10.pickle', 'wb') as to_write:
#     pickle.dump(df_lsa_model_10, to_write)
    
# with open('/Users/robertpagano/metis_data/project_5/topic_modeling/topic_dfs/df_lsa_model_15.pickle', 'wb') as to_write:
#     pickle.dump(df_lsa_model_15, to_write)
    
# with open('/Users/robertpagano/metis_data/project_5/topic_modeling/topic_dfs/df_lsa_model_20.pickle', 'wb') as to_write:
#     pickle.dump(df_lsa_model_20, to_write)
    
# with open('/Users/robertpagano/metis_data/project_5/topic_modeling/topic_dfs/df_lsa_model_25.pickle', 'wb') as to_write:
#     pickle.dump(df_lsa_model_25, to_write)
    

# with open('/Users/robertpagano/metis_data/project_5/topic_modeling/topic_dfs/df_nmf_model_10.pickle', 'wb') as to_write:
#     pickle.dump(df_nmf_model_10, to_write)
    
# with open('/Users/robertpagano/metis_data/project_5/topic_modeling/topic_dfs/df_nmf_model_15.pickle', 'wb') as to_write:
#     pickle.dump(df_nmf_model_15, to_write)
    
# with open('/Users/robertpagano/metis_data/project_5/topic_modeling/topic_dfs/df_nmf_model_20.pickle', 'wb') as to_write:
#     pickle.dump(df_nmf_model_20, to_write)
    
# with open('/Users/robertpagano/metis_data/project_5/topic_modeling/topic_dfs/df_nmf_model_25.pickle', 'wb') as to_write:
#     pickle.dump(df_nmf_model_25, to_write)
    
    

    
